In [1060]:
import torch
import pickle
from torch import nn,optim
import torch.nn.functional
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os as os
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,multilabel_confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from scipy.stats import kurtosis,skew
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
import scipy.stats
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from imblearn.over_sampling import SMOTE


from sklearn.model_selection import cross_val_score
from collections import OrderedDict

#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)


<h1>Train data prep</h1>

una osservazione ogni secondo <br>
1200 osservazioni per dataframe<br>
Le colonne P1-P7 sono pressioni nei punti di misurazione<br>

In [1061]:
path_dataset= '/Users/alexdediu/git-projects/Data-Challenge-PHM-Asia-Pacific-2023/dataset'
path_dataframes="/Users/alexdediu/git-projects/Data-Challenge-PHM-Asia-Pacific-2023/dataframes"
path_data = "/data"
path_labels = "/labels_spacecraft.xlsx"

def create_dataframe(name):
    dfs_data = []
    filenames = sorted(os.listdir(f"{path_dataset}/{name}/data"))
    for case,filename in enumerate(filenames):
        if filename.endswith('.csv'):
            percorso_file = os.path.join(f"{path_dataset}/{name}/data", filename)
            df = pd.read_csv(percorso_file)
            df["Case"]=case+1
            
            dfs_data.append(df)

    df_data = pd.concat(dfs_data, ignore_index=True)
    return df_data,dfs_data

def create_df_labels(name,filename):
    df = pd.read_excel(f"{path_dataset}/{name}/{filename}.xlsx",header=1)
    if name == "train":
        df = df.rename(columns={"Unnamed: 0":"Case","Unnamed: 1":"Spacecraft","Unnamed: 2":"Condition"})
    elif name == "test":
        df = df.rename(columns={"Case#":"Case","Spacecraft#":"Spacecraft"})
        
    return df


In [1062]:
def prepare_df_labels(df_labels):
    condition_map = {'Normal':1,'Fault':2,'Anomaly':3}
    bp_map = {'No':0,'Yes':1}
    df_labels["Condition"] = df_labels["Condition"].map(condition_map)
    bp_columns = [col for col in df_labels.columns if ("BP" in col) or("BV" in col)]

    def map_condition(value):
        if value == 1:
            return 0
        elif value in [2, 3]:
            return 1
        else:
            return None  

    df_labels['binary_condition'] = df_labels['Condition'].apply(map_condition)

    for col in bp_columns:
        df_labels[col] = df_labels[col].map(bp_map)
        
    return df_labels


In [1063]:
df_train,dfs_train = create_dataframe("train")
df_test,dfs_test = create_dataframe("test")

df_labels_train = create_df_labels("train","labels")
df_labels_test = create_df_labels("test","labels_spacecraft")
df_labels_train = prepare_df_labels(df_labels_train)

In [1064]:
df_labels_train.head()

,Case,Spacecraft,Condition,SV1,SV2,SV3,SV4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,BV1,binary_condition
0,1,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0
1,2,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0
2,3,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0
3,4,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0
4,5,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0


In [1065]:
with open('dfs_train_list.pkl', 'wb') as file:
    pickle.dump(dfs_train, file)

with open('dfs_test_list.pkl', 'wb') as file:
    pickle.dump(dfs_test, file)



df_labels_train.to_csv(f"{path_dataframes}/df_labels_train.csv")
df_labels_test.to_csv(f"{path_dataframes}/df_labels_test.csv")
# df_train.to_csv(f"{path_dataframes}/df_train")

In [1066]:
# scaler= MinMaxScaler()
# var = scaler.fit_transform(np.array(var).reshape(-1,1))
# var = var.ravel().tolist()
# print(var)
# fig = go.Figure(data=go.Scatter(x=list(range(len(var))), y=var, mode='markers'))

# # Aggiunta di titolo e label agli assi
# fig.update_layout(title='Scatter Plot Semplice',
#                   xaxis_title='X',
#                   yaxis_title='Y')

# # Visualizzazione del grafico
# fig.show()

In [1067]:
# dfs_data

# Analisi spettrale.
calcolo FFT per ogni serie temporale (P1-P7) di ogni caso (1-177)

eliminazione della frequenza 0 (componente DC).

fftfreq calcola le frequenze associate ai coefficienti ottenuti dalla fft.

il plot seguente ha sull'asse x le frequenze e sull'asse y la fft.

per la ftt basta la parte reale o bisogna mantenere anche la fase (parte immaginaria)?


In [1068]:
sampling_rate = 1000  # 1 kHz

signal_length_ms = 1200

In [1069]:

# def calculate_spectral_features(freqs,spectrum):

#     signal_power = np.sum(spectrum)
#     noise_power = np.sum(spectrum[freqs > 0.5 * sampling_rate])
#     distortion_power = signal_power - noise_power 

#     epsilon = 1e-10
#     snr = 10 * np.log10(signal_power / (noise_power + epsilon))

#     sinad = 10 * np.log10(signal_power / (noise_power + distortion_power + epsilon))

#     # peak_amplitude = np.max(time_series)

#     band_power = np.sum(spectrum[(freqs >= 0) & (freqs <= max(range(len(freq))))])

#     peak_value = np.max(spectrum)

#     peak_freq = freqs[np.argmax(spectrum)]

#     mean_power_spectrum = np.mean(spectrum)
   

#     sum_power_spectrum = np.sum(spectrum)
#     std_spectrum = np.std(spectrum)

#     return sinad, snr, band_power, peak_value, peak_freq, mean_power_spectrum, sum_power_spectrum,std_spectrum

# def calculate_additional_spectral_features(freqs,spectrum):
    

#     harmonic_amplitude_ratio = np.max(spectrum) / np.mean(spectrum)
#     main_harmonic_bandwidth = freqs[np.argmax(spectrum)] - freqs[np.argmin(spectrum)]


#     return harmonic_amplitude_ratio, main_harmonic_bandwidth


# dfs_freq = []
# dfs_fft = []
# freq_dom = []

# for df in dfs_data:
#     fft_df = pd.DataFrame()
#     fft_freq = pd.DataFrame()
#     freq_dom_col = pd.DataFrame()

#     for col in df.columns:
#         if  (col != "TIME") and (col != "Case"):
#             # print(col)
#             fft_result = np.fft.fft(df[col]).real
#             # print(len(df[col]))
#             freq = np.fft.fftfreq(len(df[col]), d=1/sampling_rate)

#             # eliminazione componente DC
#             fft_result = fft_result[1:]
#             freq = freq[1:]

#             # spettro di potenza
#             power_spectrum = np.abs(fft_result) ** 2
#             # calcolo features spettrali
#             sinad, snr, band_power, peak_value, peak_freq, mean_power_spectrum, sum_power_spectrum,std_spectrum = calculate_spectral_features(freq,power_spectrum)
#             harmonic_amplitude_ratio, main_harmonic_bandwidth = calculate_additional_spectral_features(freq,power_spectrum)


#             max_val = np.max(np.abs(fft_result))
            
#             # da spostare nelle features temporali
#             impulse_factor = max_val / np.mean(np.abs(df[col]))
#             crest_factor = max_val / np.sqrt(np.mean(np.square(df[col])))
#             # freq_dom_col["Case_fft"] = [df["Case"].iloc[0]]
#             ampl = np.abs(fft_result)
#             amplitudes_sum = np.sum(ampl)
#             prob_amplitudes = ampl / amplitudes_sum
#             entropy = -np.sum(prob_amplitudes * np.log2(prob_amplitudes + 1e-12)) # 1e-12 per evitare log(0)
#             squared = np.square(df[col])
#             mean_squared = np.mean(squared)
#             rms = np.sqrt(mean_squared)
            


#             freq_dom_col[col + "_SINAD"] = [sinad]
#             freq_dom_col[col + "_rms"] = [rms]

#             freq_dom_col[col + "_entropy"] = [entropy]

#             freq_dom_col[col + "_SNR"] = [snr]
     
#             freq_dom_col[col + "_crest_factor"] = [crest_factor]
#             freq_dom_col[col + "_impulse_factor"] = [impulse_factor]
#             freq_dom_col[col + "_BandPower"] = [band_power]
#             freq_dom_col[col + "_PeakValue"] = [peak_value]
#             freq_dom_col[col + "_PeakFreq"] = [peak_freq]
#             freq_dom_col[col + "_SumPowerSpectrum"] = [sum_power_spectrum]
#             # freq_dom_col[col + "_StdPowerSpectrum"] = [std_spectrum]

       
#             freq_dom_col[col + "_HarmonicAmplitudeRatio"] = [harmonic_amplitude_ratio]
#             freq_dom_col[col + "_MainHarmonicBandwidth"] = [main_harmonic_bandwidth]

#             fft_df[col + '_fft'] = fft_result
#             for f in freq:
#                 fft_freq[col + '_freq'] = freq
#             # fft_df["Case_fft"] = df["Case"].iloc[0]

#     dfs_freq.append(fft_freq)
#     dfs_fft.append(fft_df)
#     freq_dom.append(freq_dom_col)

# df_freq_dom = pd.concat(freq_dom, axis=0)

# print(df_freq_dom)


In [1070]:
dfs_fft

[         P1_fft     P2_fft     P3_fft     P4_fft     P5_fft     P6_fft  \
 0     -0.123507  -0.374782  -0.427204  -1.808231   0.105906   1.040863   
 1     -0.051402  -0.266674  -0.319253  -1.668430   0.193356   1.121820   
 2    -37.768564 -60.438713 -74.795698 -75.218039 -58.817075 -84.482832   
 3      0.182300   0.113201   0.136783  -1.143038   0.464277   1.572364   
 4      0.100644  -0.027089  -0.081565  -1.317471   0.315505   1.253449   
 ...         ...        ...        ...        ...        ...        ...   
 1195   0.100644  -0.027089  -0.081565  -1.317471   0.315505   1.253449   
 1196   0.182300   0.113201   0.136783  -1.143038   0.464277   1.572364   
 1197 -37.768564 -60.438713 -74.795698 -75.218039 -58.817075 -84.482832   
 1198  -0.051402  -0.266674  -0.319253  -1.668430   0.193356   1.121820   
 1199  -0.123507  -0.374782  -0.427204  -1.808231   0.105906   1.040863   
 
          P7_fft  
 0     -2.170918  
 1     -2.013958  
 2    -77.688796  
 3     -1.466425  
 4 

In [1071]:
# df_freq_normal = pd.concat(dfs_freq[:105],axis=0)
# df_freq_anomaly = pd.concat(dfs_freq[106:154],axis=0)
# df_freq_fault = pd.concat(dfs_freq[154:177],axis=0)

# df_fft_normal = pd.concat(dfs_fft[:105],axis=0)
# df_fft_anomaly = pd.concat(dfs_fft[106:154],axis=0)
# df_fft_fault = pd.concat(dfs_fft[154:177],axis=0)

### df frequenze 177x1200 = 212400

In [1072]:
# df_freq_normal

In [1073]:

# normal = dfs_fft[0]["P1_fft"]

# print(len(normal))

In [1074]:

# normal1 = dfs_fft[5]["P7_fft"]
# anomaly1 = dfs_fft[171]["P7_fft"]
# fault1 = dfs_fft[131]["P7_fft"]



In [1075]:
# frequencies = np.fft.fftfreq(signal_length_ms,1/sampling_rate)
# half_signal =int(signal_length_ms/2)

# peaks_normal = np.abs(normal1)
# peaks_anomalous = np.abs(anomaly1)
# peaks_fault = np.abs(fault1)




# peaks_normal = np.abs(normal1[:half_signal])
# peaks_anomalous = np.abs(anomaly1[:half_signal])
# peaks_fault = np.abs(fault1[:half_signal])


# plt.figure(figsize=(20, 10))

# plt.plot(frequencies[:half_signal], peaks_normal,alpha=0.6,label="normal")
# plt.title('FFT - Caso Normale')
# plt.xlabel('Frequenza (Hz)')
# plt.ylabel('Intensità')
# plt.tight_layout()


# plt.plot(frequencies[:half_signal], peaks_anomalous,alpha=0.4,label="anomaly")
# plt.title('FFT - Caso Anomaly')
# plt.xlabel('Frequenza (Hz)')
# plt.ylabel('Intensità')

# plt.tight_layout()


# plt.plot(frequencies[:half_signal], peaks_fault,alpha=0.5,label="fault")
# plt.title('FFT - Caso Fault')
# plt.xlabel('Frequenza (Hz)')
# plt.ylabel('Intensità')
# plt.tight_layout()

# plt.legend()

# plt.show()

In [1076]:
# import plotly.graph_objects as go

# for column in ["P1","P2","P3","P4","P5","P6","P7"]:
#     fig = go.Figure()

   

#     # Aggiungi le tre curve
#     fig.add_trace(go.Scatter(x=df_freq_normal[column+"_freq"], y=df_fft_normal[column+"_fft"], mode='lines', name=f'Normal', line=dict(color='blue', width=2), opacity=0.6))
#     fig.add_trace(go.Scatter(x=df_freq_anomaly[column+"_freq"], y=df_fft_anomaly[column+"_fft"], mode='lines', name='Anomaly', line=dict(color='orange', width=2), opacity=0.4))
#     fig.add_trace(go.Scatter(x=df_freq_fault[column+"_freq"], y=df_fft_fault[column+"_fft"], mode='lines', name='Fault', line=dict(color='green', width=2), opacity=0.5))

#     # Aggiungi titolo e label agli assi
#     fig.update_layout(title=f'{column}',
#                     xaxis_title='Frequenza (Hz)',
#                     yaxis_title='Ampiezza')

#     # Mostra la legenda
#     fig.update_layout(showlegend=True)

#     # Mostra il grafico
#     fig.show()


In [1077]:
# condition_map = {'Normal':1,'Fault':2,'Anomaly':3}
# bp_map = {'No':0,'Yes':1}
# df_labels = pd.read_excel(f"{path_train}/labels.xlsx",header=1)

In [1078]:
# df_labels = df_labels.drop(columns=["Unnamed: 1","Unnamed: 0","Unnamed: 2"])

In [1079]:
# df_labels.to_csv("df_labels_2.csv")

In [1080]:
# freq_dom[0]
# df_condition_2 = df_labels[df_labels["Condition"] == 2]
# df_condition_3 = df_labels[df_labels["Condition"] == 3]

In [1081]:
# condition_map = {'Normal':1,'Fault':2,'Anomaly':3}
# bp_map = {'No':0,'Yes':1}

# df_labels = df_labels.rename(columns={"Unnamed: 0":"Case","Unnamed: 1":"Spacecraft","Unnamed: 2":"Condition"})

# df_labels["Condition"] = df_labels["Condition"].map(condition_map)
# bp_columns = [col for col in df_labels.columns if ("BP" in col) or("BV" in col)]
# print(bp_columns)

# def map_condition(value):
#     if value == 1:
#         return 0
#     elif value in [2, 3]:
#         return 1
#     else:
#         return None  

# df_labels['binary_condition'] = df_labels['Condition'].apply(map_condition)

# for col in bp_columns:
#     df_labels[col] = df_labels[col].map(bp_map)

# df_labels.to_csv("df_labels.csv")
# df_labels.to_csv("df_labels_2.csv")


### 105 casi normali
### 48 casi anomali
### 24 casi fault

In [1082]:
# len(df_labels[df_labels["Condition"] == 3])

In [1083]:
# df_labels

In [1084]:
def calculate_column_statistics(column):
 
    statistics = {
        'mean': column.mean(),
        'median': column.median(),
        'std': column.std(),
        'min': column.min(),
        'max': column.max(),
        '25th_percentile': column.quantile(0.25),
        '75th_percentile': column.quantile(0.75),
        'kurtosis': kurtosis(column),
        'skewness': skew(column)
    
        # Puoi aggiungere altre statistiche a seconda delle tue esigenze
    }
    return pd.Series(statistics)

# Stats su serie temporale

In [1085]:





# dfs_temp_stats = []


# for df in dfs_data:
#     statistics_dict = OrderedDict()    
#     for col in df.columns:
#         if (col != "TIME") and (col != "Case"):
#             column_stats = calculate_column_statistics(df[col])
#             statistics_dict[col + "_mean"] = column_stats["mean"]
#             # statistics_dict[col + "_median"] = column_stats["median"]
#             # statistics_dict[col + "_std"] = column_stats["std"]
#             # statistics_dict[col + "_min"] = column_stats["min"]
#             # statistics_dict[col + "_max"] = column_stats["max"]
#             # statistics_dict[col + "_25th_percentile"] = column_stats["25th_percentile"]
#             # statistics_dict[col + "_75th_percentile"] = column_stats["75th_percentile"]
#             line_integral = np.trapz(df[col])

# # Calcolo del clearance factor
#             clearance_factor = np.max(df[col]) - np.min(df[col])
      
#             statistics_dict[col + "_Variance"] = np.var(df[col])
#             statistics_dict[col + "_LI"] = line_integral
#             statistics_dict[col + "_Cfactor"] = clearance_factor
#             # statistics_dict["Case_stats_temp"] = df["Case"].iloc[0]




         
#     statistics_df = pd.DataFrame(statistics_dict,index=[0])
#     dfs_temp_stats.append(statistics_df)



# Stats su dfs FFT

In [1086]:
# dfs_fft_stats = []
# c = 0
# for df in dfs_fft:
#     statistics_dict = OrderedDict()
#     for col in df.columns:

#         if (col != "TIME") and (col != "Case"):
#             column_stats = calculate_column_statistics(df[col])
#             statistics_dict[col + "_mean"] = column_stats["mean"]

#             statistics_dict[col + "_median"] = column_stats["median"]
#             statistics_dict[col + "_std"] = column_stats["std"]
#             statistics_dict[col + "_min"] = column_stats["min"]
#             statistics_dict[col + "_max"] = column_stats["max"]
#             statistics_dict[col + "_25th_percentile"] = column_stats["25th_percentile"]
#             statistics_dict[col + "_75th_percentile"] = column_stats["75th_percentile"]
#             statistics_dict[col + "_kurtosis"] = column_stats["kurtosis"]
#             statistics_dict[col + "_skewness"] = column_stats["skewness"]
#             statistics_dict[col + "_Variance"] = np.var(df[col])
#             statistics_dict[col + "_LI"] = line_integral
#             statistics_dict[col + "_Cfactor"] = clearance_factor
            
            
#             # statistics_dict["Case_stats_fft"] = df["Case_fft"].iloc[0]

#     c = c+1
#     statistics_df_fft = pd.DataFrame(statistics_dict,index=[0])
#     dfs_fft_stats.append(statistics_df_fft)




In [1087]:
# dfs_temp_stats = pd.concat(dfs_temp_stats,axis=0)
# dfs_fft_stats= pd.concat(dfs_fft_stats,axis=0)

In [1088]:
# scaler_FFT = MinMaxScaler()
# dfs_fft_stats_scaled = scaler_FFT.fit_transform(dfs_fft_stats)
# dfs_fft_stats = pd.DataFrame(dfs_fft_stats_scaled, columns=dfs_fft_stats.columns)


In [1089]:
# scaler_freq = MinMaxScaler()
# dfs_freq_temp= scaler_freq.fit_transform(df_freq_dom)
# df_freq_dom= pd.DataFrame(dfs_freq_temp, columns=df_freq_dom.columns)

In [1090]:
# dfs_fft_stats

In [1091]:
# scaler = MinMaxScaler()
# dfs_temp_stats_scaled = scaler.fit_transform(dfs_temp_stats)

# # Creazione di un nuovo DataFrame con le features scalate
# dfs_temp_stats = pd.DataFrame(dfs_temp_stats_scaled, columns=dfs_temp_stats.columns)


In [1092]:
# dfs_temp_stats

### Concat di tutti i dfs

In [1093]:
# df_freq= pd.concat(dfs_freq,axis=1)

# df_freq

In [1094]:
# dfs_temp_stats = dfs_temp_stats.reset_index(drop=True)
# dfs_fft_stats = dfs_fft_stats.reset_index(drop=True)
# df_freq_dom = df_freq_dom.reset_index(drop=True)

In [1095]:

# df_features = pd.concat([dfs_temp_stats,dfs_fft_stats,df_freq_dom],axis=1)

In [1096]:



# df_features["Condition"] = df_labels["Condition"]
df_features["binary_condition"] = pd.Series(df_labels["binary_condition"].values)

In [1097]:
df_features

,P5_HarmonicAmplitudeRatio,P5_rms,P5_Variance,P1_Cfactor,P1_Variance,P1_rms,P1_HarmonicAmplitudeRatio,P2_Variance,P2_rms,P6_HarmonicAmplitudeRatio,...,P3_LI,P3_fft_mean,P3_mean,P2_SINAD,P2_fft_skewness,P1_fft_25th_percentile,P7_HarmonicAmplitudeRatio,P5_SINAD,Spacecraft,binary_condition
0,0.404716,0.908139,0.989111,0.835385,0.997566,0.957028,0.500515,0.742221,0.587077,0.535152,...,0.246580,0.753381,0.246619,0.8,0.467707,0.396821,0.558522,0.8,1,0
1,0.386983,0.957102,0.970816,0.856063,0.999072,0.970137,0.496295,0.741918,0.572730,0.534254,...,0.251591,0.748367,0.251633,0.8,0.461777,0.356142,0.577813,0.8,1,0
2,0.385712,0.908687,0.942070,0.884633,0.997855,0.979246,0.492228,0.752047,0.614115,0.516265,...,0.232074,0.767832,0.232168,0.8,0.453730,0.394658,0.414686,0.8,1,0
3,0.384052,0.937148,0.964525,0.884439,0.995734,0.969917,0.492075,0.751956,0.622608,0.509841,...,0.235775,0.764141,0.235859,0.8,0.448720,0.429939,0.554054,0.8,1,0
4,0.382267,0.961120,0.975974,0.873784,0.996313,0.974559,0.490276,0.745000,0.595336,0.505803,...,0.248419,0.751535,0.248465,0.8,0.445623,0.428040,0.537570,0.8,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,1.000000,0.205255,0.504254,0.598945,0.585004,0.245719,1.000000,1.000000,1.000000,0.687747,...,0.227071,0.775071,0.224929,1.0,0.451714,0.390637,0.923026,0.8,3,1
173,0.697196,0.719323,0.781425,0.428516,0.583065,0.138386,0.717433,0.647391,0.448477,0.730663,...,0.327667,0.671671,0.328329,0.8,0.970296,0.791386,0.320495,1.0,3,1
174,0.438246,0.344787,0.577233,0.737837,0.531799,0.024519,0.362797,0.576285,0.348801,0.561968,...,0.321253,0.681690,0.318310,0.4,0.702085,0.584715,0.260117,0.4,3,1
175,0.000000,0.204130,0.497790,0.636050,0.500953,0.139861,0.000000,0.569685,0.239404,0.000000,...,0.358124,0.644517,0.355483,0.2,0.868546,0.693178,0.073649,0.2,3,1


In [1098]:
nan_columns = df_features.columns[df_features.isna().any()].tolist()

# Print the columns that contain NaN values
if nan_columns:
    print("Columns containing NaN values:")
    for col in nan_columns:
        print(col)
else:
    print("No columns contain NaN values.")

No columns contain NaN values.


In [1099]:
df_y = pd.DataFrame()
df_y["binary_condition"] = df_features["binary_condition"]

In [1100]:
import pandas as pd
from sklearn.feature_selection import SelectFromModel, f_classif
from sklearn.linear_model import LogisticRegression
X = df_features.drop(["binary_condition"], axis=1)
y = df_features['binary_condition']



f_values, p_values = f_classif(X, y)

results = pd.DataFrame({'Feature': X.columns, 'F_Value': f_values, 'P_Value': p_values})

results = results.sort_values(by='F_Value', ascending=False)

top_50_features = results.head(20)
features = top_50_features["Feature"].values
features=np.concatenate([features, ["binary_condition"]])

print(results["Feature"].values)

['P5_HarmonicAmplitudeRatio' 'P5_rms' 'P5_Variance' 'P1_Cfactor'
 'P1_Variance' 'P1_rms' 'P1_HarmonicAmplitudeRatio' 'P2_Variance' 'P2_rms'
 'P6_HarmonicAmplitudeRatio' 'P5_fft_25th_percentile' 'P1_fft_median'
 'P3_LI' 'P3_fft_mean' 'P3_mean' 'P2_SINAD' 'P2_fft_skewness'
 'P1_fft_25th_percentile' 'P7_HarmonicAmplitudeRatio' 'P5_SINAD'
 'Spacecraft']


In [1101]:
df_features = df_features[features]

In [1102]:
df_features

,P5_HarmonicAmplitudeRatio,P5_rms,P5_Variance,P1_Cfactor,P1_Variance,P1_rms,P1_HarmonicAmplitudeRatio,P2_Variance,P2_rms,P6_HarmonicAmplitudeRatio,...,P1_fft_median,P3_LI,P3_fft_mean,P3_mean,P2_SINAD,P2_fft_skewness,P1_fft_25th_percentile,P7_HarmonicAmplitudeRatio,P5_SINAD,binary_condition
0,0.404716,0.908139,0.989111,0.835385,0.997566,0.957028,0.500515,0.742221,0.587077,0.535152,...,0.947263,0.246580,0.753381,0.246619,0.8,0.467707,0.396821,0.558522,0.8,0
1,0.386983,0.957102,0.970816,0.856063,0.999072,0.970137,0.496295,0.741918,0.572730,0.534254,...,0.881227,0.251591,0.748367,0.251633,0.8,0.461777,0.356142,0.577813,0.8,0
2,0.385712,0.908687,0.942070,0.884633,0.997855,0.979246,0.492228,0.752047,0.614115,0.516265,...,0.878493,0.232074,0.767832,0.232168,0.8,0.453730,0.394658,0.414686,0.8,0
3,0.384052,0.937148,0.964525,0.884439,0.995734,0.969917,0.492075,0.751956,0.622608,0.509841,...,0.850933,0.235775,0.764141,0.235859,0.8,0.448720,0.429939,0.554054,0.8,0
4,0.382267,0.961120,0.975974,0.873784,0.996313,0.974559,0.490276,0.745000,0.595336,0.505803,...,0.867672,0.248419,0.751535,0.248465,0.8,0.445623,0.428040,0.537570,0.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,1.000000,0.205255,0.504254,0.598945,0.585004,0.245719,1.000000,1.000000,1.000000,0.687747,...,0.170685,0.227071,0.775071,0.224929,1.0,0.451714,0.390637,0.923026,0.8,1
173,0.697196,0.719323,0.781425,0.428516,0.583065,0.138386,0.717433,0.647391,0.448477,0.730663,...,0.396663,0.327667,0.671671,0.328329,0.8,0.970296,0.791386,0.320495,1.0,1
174,0.438246,0.344787,0.577233,0.737837,0.531799,0.024519,0.362797,0.576285,0.348801,0.561968,...,0.509101,0.321253,0.681690,0.318310,0.4,0.702085,0.584715,0.260117,0.4,1
175,0.000000,0.204130,0.497790,0.636050,0.500953,0.139861,0.000000,0.569685,0.239404,0.000000,...,0.390568,0.358124,0.644517,0.355483,0.2,0.868546,0.693178,0.073649,0.2,1


In [1103]:
# df_features

In [1104]:
df_features["Spacecraft"] = df_labels["Spacecraft"]

In [1105]:
df_features

,P5_HarmonicAmplitudeRatio,P5_rms,P5_Variance,P1_Cfactor,P1_Variance,P1_rms,P1_HarmonicAmplitudeRatio,P2_Variance,P2_rms,P6_HarmonicAmplitudeRatio,...,P3_LI,P3_fft_mean,P3_mean,P2_SINAD,P2_fft_skewness,P1_fft_25th_percentile,P7_HarmonicAmplitudeRatio,P5_SINAD,binary_condition,Spacecraft
0,0.404716,0.908139,0.989111,0.835385,0.997566,0.957028,0.500515,0.742221,0.587077,0.535152,...,0.246580,0.753381,0.246619,0.8,0.467707,0.396821,0.558522,0.8,0,1
1,0.386983,0.957102,0.970816,0.856063,0.999072,0.970137,0.496295,0.741918,0.572730,0.534254,...,0.251591,0.748367,0.251633,0.8,0.461777,0.356142,0.577813,0.8,0,1
2,0.385712,0.908687,0.942070,0.884633,0.997855,0.979246,0.492228,0.752047,0.614115,0.516265,...,0.232074,0.767832,0.232168,0.8,0.453730,0.394658,0.414686,0.8,0,1
3,0.384052,0.937148,0.964525,0.884439,0.995734,0.969917,0.492075,0.751956,0.622608,0.509841,...,0.235775,0.764141,0.235859,0.8,0.448720,0.429939,0.554054,0.8,0,1
4,0.382267,0.961120,0.975974,0.873784,0.996313,0.974559,0.490276,0.745000,0.595336,0.505803,...,0.248419,0.751535,0.248465,0.8,0.445623,0.428040,0.537570,0.8,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,1.000000,0.205255,0.504254,0.598945,0.585004,0.245719,1.000000,1.000000,1.000000,0.687747,...,0.227071,0.775071,0.224929,1.0,0.451714,0.390637,0.923026,0.8,1,3
173,0.697196,0.719323,0.781425,0.428516,0.583065,0.138386,0.717433,0.647391,0.448477,0.730663,...,0.327667,0.671671,0.328329,0.8,0.970296,0.791386,0.320495,1.0,1,3
174,0.438246,0.344787,0.577233,0.737837,0.531799,0.024519,0.362797,0.576285,0.348801,0.561968,...,0.321253,0.681690,0.318310,0.4,0.702085,0.584715,0.260117,0.4,1,3
175,0.000000,0.204130,0.497790,0.636050,0.500953,0.139861,0.000000,0.569685,0.239404,0.000000,...,0.358124,0.644517,0.355483,0.2,0.868546,0.693178,0.073649,0.2,1,3


In [1106]:
df_features = df_features.drop(columns="binary_condition")

# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(df_features, df_y["binary_condition"])

# df_features = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled, columns=['binary_condition'])], axis=1)


In [1107]:
# scaler_features = MinMaxScaler()


# df_features_scaled = scaler_features.fit_transform(df_features)

# df_features = pd.DataFrame(df_features_scaled, columns=df_features.columns)


# df_features

In [1108]:
df_y

,binary_condition
0,0
1,0
2,0
3,0
4,0
...,...
172,1
173,1
174,1
175,1


In [1109]:
# len(df_features[df_features["binary_condition"] == 1])

In [1110]:

df_y.to_csv("df_labels.csv")


In [1111]:
df_features

,P5_HarmonicAmplitudeRatio,P5_rms,P5_Variance,P1_Cfactor,P1_Variance,P1_rms,P1_HarmonicAmplitudeRatio,P2_Variance,P2_rms,P6_HarmonicAmplitudeRatio,...,P1_fft_median,P3_LI,P3_fft_mean,P3_mean,P2_SINAD,P2_fft_skewness,P1_fft_25th_percentile,P7_HarmonicAmplitudeRatio,P5_SINAD,Spacecraft
0,0.404716,0.908139,0.989111,0.835385,0.997566,0.957028,0.500515,0.742221,0.587077,0.535152,...,0.947263,0.246580,0.753381,0.246619,0.8,0.467707,0.396821,0.558522,0.8,1
1,0.386983,0.957102,0.970816,0.856063,0.999072,0.970137,0.496295,0.741918,0.572730,0.534254,...,0.881227,0.251591,0.748367,0.251633,0.8,0.461777,0.356142,0.577813,0.8,1
2,0.385712,0.908687,0.942070,0.884633,0.997855,0.979246,0.492228,0.752047,0.614115,0.516265,...,0.878493,0.232074,0.767832,0.232168,0.8,0.453730,0.394658,0.414686,0.8,1
3,0.384052,0.937148,0.964525,0.884439,0.995734,0.969917,0.492075,0.751956,0.622608,0.509841,...,0.850933,0.235775,0.764141,0.235859,0.8,0.448720,0.429939,0.554054,0.8,1
4,0.382267,0.961120,0.975974,0.873784,0.996313,0.974559,0.490276,0.745000,0.595336,0.505803,...,0.867672,0.248419,0.751535,0.248465,0.8,0.445623,0.428040,0.537570,0.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,1.000000,0.205255,0.504254,0.598945,0.585004,0.245719,1.000000,1.000000,1.000000,0.687747,...,0.170685,0.227071,0.775071,0.224929,1.0,0.451714,0.390637,0.923026,0.8,3
173,0.697196,0.719323,0.781425,0.428516,0.583065,0.138386,0.717433,0.647391,0.448477,0.730663,...,0.396663,0.327667,0.671671,0.328329,0.8,0.970296,0.791386,0.320495,1.0,3
174,0.438246,0.344787,0.577233,0.737837,0.531799,0.024519,0.362797,0.576285,0.348801,0.561968,...,0.509101,0.321253,0.681690,0.318310,0.4,0.702085,0.584715,0.260117,0.4,3
175,0.000000,0.204130,0.497790,0.636050,0.500953,0.139861,0.000000,0.569685,0.239404,0.000000,...,0.390568,0.358124,0.644517,0.355483,0.2,0.868546,0.693178,0.073649,0.2,3


In [1112]:
conditions= df_labels["Condition"].unique()

print(conditions)

[1 2 3]


In [1113]:

df_features.to_csv("df_features.csv")
